In [1]:
#Import Libraries
import os
import pandas as pd
from gurobipy import Model, GRB, quicksum, max_

In [2]:
df_rare = pd.read_csv("https://raw.githubusercontent.com/mlssorarescout/sorareanalytics/main/lineup_optimization/mls_sorare_rare.csv")
df_rare.head()

,player,Club,Score,position,Rarity,Usd,ETH
0,S. Berhalter - Vancouver Whitecaps FC,Vancouver Whitecaps FC,54.1,3,rare,162.633333,5.420000e+16
1,J. Brown - Vancouver Whitecaps FC,Vancouver Whitecaps FC,43.4,2,rare,44.030000,2.090000e+16
2,C. Cropper - Vancouver Whitecaps FC,Vancouver Whitecaps FC,23.7,1,rare,331.106667,1.680000e+17
3,F. Jungwirth - Vancouver Whitecaps FC,Vancouver Whitecaps FC,36.0,2,rare,35.920000,1.270000e+16
4,M. Campagna - Vancouver Whitecaps FC,Vancouver Whitecaps FC,0.0,3,rare,86.075000,3.530000e+16


In [3]:
indices = df_rare.player
points = dict(zip(indices, df_rare.Score))
cost = dict(zip(indices, df_rare.Usd))
S = 460

m = Model();

y = m.addVars(df_rare.player, vtype=GRB.BINARY, name="y")
m.setObjective(quicksum(cost[i]*y[i] for i in indices), GRB.MINIMIZE)

player_position_map = list(zip(df_rare.player, df_rare.position))
player_map = list(df_rare.player)

#add position constraints
m.addConstr(quicksum([y[i] for i, position in player_position_map if position==1])==1);

m.addConstr(quicksum([y[i] for i, position in player_position_map if position==2])>=1);
m.addConstr(quicksum([y[i] for i, position in player_position_map if position==2])<=2);

m.addConstr(quicksum([y[i] for i, position in player_position_map if position==3])>=1);
m.addConstr(quicksum([y[i] for i, position in player_position_map if position==3])<=2);

m.addConstr(quicksum([y[i] for i, position in player_position_map if position==4])>=1);
m.addConstr(quicksum([y[i] for i, position in player_position_map if position==4])<=2);

#add number of players in lineup constraint 
m.addConstr(quicksum([y[i] for i in indices])==5);

#add constraint that points + points bonus must be greater than S
m.addConstr(quicksum((points[i]+points[i]*.05)*y[i] for i in indices) >= S, name="points");

m.optimize()

results = pd.DataFrame()
                                          
for v in m.getVars():
    if v.x > 1e-6:
        results = results.append(df_rare.iloc[v.index][['player','position','Score','Usd']])
        print(v.varName, v.x)

print('Cost in USD $', m.objVal)

Restricted license - for non-production use only - expires 2023-10-25
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 9 rows, 709 columns and 2421 nonzeros
Model fingerprint: 0x899d6f3a
Variable types: 0 continuous, 709 integer (709 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [4e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+02]
Presolve removed 0 rows and 81 columns
Presolve time: 0.00s
Presolved: 9 rows, 628 columns, 2230 nonzeros
Variable types: 0 continuous, 628 integer (628 binary)

Root relaxation: objective 1.588314e+03, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1588.31379    0    2          - 1588.31379      -     -    0s
H    0     0      

In [4]:
results.head()

,player,position,Score,Usd
134,D. Rubio - Colorado Rapids,4.0,82.0,520.036667
377,S. Breza - CF Montreal,1.0,68.6,1023.120000
474,S. Nealis - New York Red Bulls,2.0,96.4,163.953333
532,S. Blanco - Portland Timbers,3.0,100.0,117.913333
543,D. CharÃ¡ - Portland Timbers,3.0,95.9,104.790000
